In [1]:
!pip install librosa
!pip install pandas

In [2]:
import librosa
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split

In [3]:
#from google.colab import drive
#drive.mount('/content/drive')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device}.')

Using cuda.


In [4]:
arousal = pd.read_csv('annotation/arousal.csv').to_numpy()[:,1:58]
valence = pd.read_csv('annotation/valence.csv').to_numpy()[:,1:58]
song_ids_list = pd.read_csv('annotation/valence.csv').to_numpy()[:,0].tolist()
song_ids_list = list(map(int, song_ids_list))

In [5]:
arousal.shape

(1802, 57)

In [6]:
def stratified_sample_by_cluster(data, test_ratio):
    unique_clusters = np.unique(data[:, -1])
    test_indices = np.array([], dtype=int)
    train_indices = np.array([], dtype=int)

    for cluster in unique_clusters:
        cluster_indices = np.where(data[:, -1] == cluster)[0]
        test_size = int(len(cluster_indices) * test_ratio)
        test_cluster_indices = np.random.choice(cluster_indices, size=test_size, replace=False)
        train_cluster_indices = np.array(list(set(cluster_indices) - set(test_cluster_indices)), dtype=int)
        test_indices = np.concatenate([test_indices, test_cluster_indices])
        train_indices = np.concatenate([train_indices, train_cluster_indices])

    return data[train_indices], data[test_indices]

def sampling_K_Mean(Y,test_ratio):
    kmeans = KMeans(n_clusters=10, random_state=42).fit(Y[:,1:])
    clusters = kmeans.labels_
    data_with_clusters = np.column_stack((Y, clusters))
    train_data, test_data = stratified_sample_by_cluster(data_with_clusters, test_ratio=test_ratio)
    X_train = train_data[:, :-1]
    X_test = test_data[:, :-1]
    
    return X_train[:,0].astype(int),X_test[:,0].astype(int)

In [7]:
def Audio_seq_generator(song_ids_list,song_ids,arousal,valence,n_mels,n_MFCCs):
    X=[]
    #X_F=[]
    Y=[]
    for song_id in song_ids:
        #feature = pd.read_csv(f'features/{song_id}.csv',sep=';').apply(pd.to_numeric, errors='coerce').to_numpy()[31:88,:]
        #if feature.shape != (57,261):
            #print(song_id,feature.shape)
        #else:
            audio,sr = librosa.load(f"Audio/{song_id}.mp3",sr=44100)
            audio=audio[:sr*45]
            
            S = librosa.feature.melspectrogram(y=audio,sr=sr,n_mels=n_mels,hop_length=441)
            S = S[:,:4500-45]
            S_dB = librosa.power_to_db(S, ref=np.max)
            SS = S_dB.reshape(n_mels,4500-45)
            
            zcr = librosa.feature.zero_crossing_rate(audio,hop_length=441)
            zcr = zcr[:,:4500-45]
    
            mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_MFCCs,hop_length=441)
            mfccs = mfccs[:,:4500-45]
            #print(SS.shape,zcr.shape)
            F = np.vstack((SS,zcr,mfccs))
            X.append(F)
            
            Y_index = song_ids_list.index(song_id)
            Y_seq = torch.tensor(np.stack((arousal[Y_index,:],valence[Y_index,:]),axis=1))
            Y_seq = [Y_seq[i*10:i*10+20].mean(dim=0)  for i in range(4)]
            Y_seq = torch.stack(Y_seq)
            Y.append(Y_seq) 
            
            
            #feature = torch.tensor(feature)
            #feature = [feature[i*10:i*10+10].mean(dim=0)  for i in range(5)]
            #feature = torch.stack(feature)
            #X_F.append(feature)
    X=torch.tensor(np.stack(X,axis=0))
    #X_F=torch.tensor(np.stack(X_F,axis=0))
    Y=torch.tensor(np.stack(Y,axis=0))
    X = [X[:,:,i*500+1500:i*500+2500] for i in range(4)]
    X = torch.tensor(np.stack(X,axis=1))
    X = [X[:,:,:,i*100:i*100+100] for i in range(10)]
    X = torch.tensor(np.stack(X,axis=2))
    return X,Y#X_F,Y

In [8]:
X, Y=Audio_seq_generator(song_ids_list,song_ids_list[0:900],arousal,valence,26,26)

In [9]:
X2, Y2=Audio_seq_generator(song_ids_list,song_ids_list[900:],arousal,valence,26,26)

In [10]:
X=torch.cat((X,X2),dim=0)
Y=torch.cat((Y,Y2),dim=0)

In [11]:
X=X.reshape(X.shape[0]*X.shape[1],X.shape[2],X.shape[3],X.shape[4])
Y=Y.reshape(Y.shape[0]*Y.shape[1],2)

In [12]:
print(X.shape,Y.shape)

torch.Size([7208, 10, 53, 100]) torch.Size([7208, 2])


In [13]:
from sklearn.model_selection import StratifiedShuffleSplit
labels = torch.zeros(Y.shape[0], dtype=torch.int64)  # Create a tensor for labels
labels[(Y[:,0] > 0) & (Y[:,1] > 0)] = 0
labels[(Y[:,0] > 0) & (Y[:,1] < 0)] = 1
labels[(Y[:,0] < 0) & (Y[:,1] > 0)] = 2
labels[(Y[:,0] < 0) & (Y[:,1] < 0)] = 3
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
train_idx, test_idx = next(sss.split(X, labels))

# 使用索引从合并的张量中获取训练集和测试集
train_X = X[train_idx]
train_Y = Y[train_idx]
test_X = X[test_idx]
test_Y = Y[test_idx]

# 检查结果
print("Training set X shape:", train_X.shape)
print("Training set Y shape:", train_Y.shape)
print("Test set X shape:", test_X.shape)
print("Test set Y shape:", test_Y.shape)

Training set X shape: torch.Size([5045, 10, 53, 100])
Training set Y shape: torch.Size([5045, 2])
Test set X shape: torch.Size([2163, 10, 53, 100])
Test set Y shape: torch.Size([2163, 2])


In [14]:
torch.save(train_X,'Data/X_train_10s_all_2626_100.pt')
torch.save(train_Y,'Data/Y_train_10s_all_2626_100.pt')
torch.save(test_X,'Data/X_val_10s_all_2626_100.pt')
torch.save(test_Y,'Data/Y_val_10s_all_2626_100.pt')